In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%%time 

limit = 80
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, tokenizer=tokenizer, n_frames=600)
my_dataloader = data.DataLoader(training_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

 ===== They are 800 transcriptions in the dataset. ===== 
CPU times: user 435 ms, sys: 140 ms, total: 575 ms
Wall time: 575 ms


In [4]:
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=64).to(device)
decoder = DecoderATTRNN1(vocab_tar_size, embedding_dim_target, dec_units=64, batch_sz=params['batch_size'], 
                         hidden_size=64, method='super_head').to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [ ]:
# Train the model
global_trainer(10, my_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'], tokenizer )

 ====== ====== ====== ====== ====== ======
      The model has 1843311 parameters
 ====== ====== ====== ====== ====== ======


Epoch 4: 100%|████████████████████| 4/4 [00:08<00:00,  2.11s/it, Loss 6.5324]

In [ ]:
mfccs, _ = training_set[0]

In [ ]:
greedy_decode(mfccs.unsqueeze(0), 40, encoder, decoder, tokenizer, device)